In [1]:
from d2c.descriptors import D2C, DataLoader


In [2]:
#suppress warnings #TODO: handle
import warnings
warnings.filterwarnings("ignore")

In [3]:
dataloader = DataLoader()
dataloader.from_pickle('example/synthetic_data.pkl')
observations = dataloader.get_observations()
dags = dataloader.get_dags()

d2c = D2C(observations=observations,
        dags=dags, 
        seed=42,
        n_jobs=40,
        full=True)

d2c.initialize()

In [4]:
df = d2c.get_descriptors_df()

In [5]:
df.shape

(210000, 92)

In [7]:
functions_used = [
    "linear",
    "sigmoid",
    "interaction",
    "poly_deg_1_2",
    "poly_deg_2_3",
    "nonlinear_<built-in function sin>",
    "nonlinear_<built-in function sqrt>",
    "nonlinear_<built-in function tanh>",
]

df['function_id'] = df['graph_id'] // 40
df.replace({'function_id': {i:function for i,function in enumerate(functions_used)}}, inplace=True)
df

,graph_id,edge_source,edge_dest,is_causal,coeff_cause,coeff_eff,HOC_3_1,HOC_1_2,HOC_2_1,HOC_1_3,...,mbe_mbe_eff_q3,mbe_mbe_eff_q4,mbe_mbe_eff_q5,mbe_mbe_eff_q6,n_samples,n_features,n_features/n_samples,skewness_ca,skewness_ef,function_id
0,0,X0,X1,False,-0.261683,-0.004412,1.928542,0.008679,0.007844,2.063397,...,0.0,0.0,0.0,0.0,250,6,0.024,0.023486,0.011552,linear
1,0,X0,X2,True,0.094991,0.034480,2.040746,0.022221,0.024236,2.044276,...,0.0,0.0,0.0,0.0,250,6,0.024,0.023486,0.018137,linear
2,0,X0,X3,True,1.227406,0.200963,2.069547,0.020006,0.022467,2.066054,...,0.0,0.0,0.0,0.0,250,6,0.024,0.023486,0.016252,linear
3,0,X0,U0,False,-1.504032,-0.004314,-0.029907,0.119946,-0.011614,-0.094461,...,0.0,0.0,0.0,0.0,250,6,0.024,0.023486,-0.082144,linear
4,0,X0,U1,False,0.334981,0.250016,1.935791,-0.005974,0.009974,1.822893,...,0.0,0.0,0.0,0.0,250,6,0.024,0.023486,-0.025681,linear
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,6999,U1,X0,True,0.139425,0.316423,0.673873,-0.547832,-0.035525,2.467208,...,0.0,0.0,0.0,0.0,250,6,0.024,0.032110,-1.810055,174
209996,6999,U1,X1,False,0.017814,-0.289195,0.064140,0.030773,0.019830,0.024448,...,0.0,0.0,0.0,0.0,250,6,0.024,0.032110,-0.464651,174
209997,6999,U1,X2,False,-0.006069,-0.033508,0.402897,-0.476167,-0.029211,2.158774,...,0.0,0.0,0.0,0.0,250,6,0.024,0.032110,-1.517707,174
209998,6999,U1,X3,False,0.016356,0.094935,0.424275,-0.250038,0.013798,0.820428,...,0.0,0.0,0.0,0.0,250,6,0.024,0.032110,-0.332135,174


In [8]:
df.to_csv('example/descriptors.csv', index=False)